## Using an LSTM char model for email generation
Code for **Introduction to LSTMs** <br/>
Compiled by: Vijai Kasthuri Rangan <br/>
Checkout: https://www.tensorflow.org/tutorials/sequences/text_generation <br/>
http://karpathy.github.io/2015/05/21/rnn-effectiveness/ <br>
v1

In [ ]:
import tensorflow as tf

In [ ]:
tf.executing_eagerly()

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()

import numpy as np
import os
import time

Read extracted email content <br/>
Replace this to be any text content that you would want the char model to learn from <br/>

In [ ]:
text = open("my_emails2.txt").read()

Clean the data more to remove any unwanted characters <br/>
Another idea would be to weigh down the probabilities associated with the special characters to be less at the time of prediction

In [ ]:
text=text.replace('-', '')

In [ ]:
print ('Length of text: {} characters'.format(len(text)))

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

In [ ]:
np.array(vocab)

Map each character to an index and vice versa

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int

In [ ]:
np.unique(text_as_int, return_counts=True)

In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

In [ ]:
#For each input sequence, the corresponding targets contain the same length of text, 
#except shifted one character to the right

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
    ;
    #print(idx2char[i.numpy()])


In [ ]:
examples_per_epoch

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
                                 tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
    return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

In [ ]:
model.summary()

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))



In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_final'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=15

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
        if (idx2char[predicted_id] == '@'):
            break;
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
#print(generate_text(model, start_string=u"INTRODUCTION"))

In [ ]:
print(generate_text(model, start_string=u"Hi "))

In [ ]:
#Remove some characters from vocabulary that do not make sense
#Include a stop character in the vocabulary that can be used to limit the text generated from the model
#Print text from the model after every 5 iterations
#Modify the char model to word model
#Add Attention to the model

In [ ]:
## Check using other model checkpoints

In [ ]:
def print_multi_epoch_output(model, EPOCHS):
    for i in range(0, EPOCHS, 4):
        model.load_weights('./training_checkpoints_final/ckpt_'+str(i+1))

        model.build(tf.TensorShape([1, None]))

        print('Model output from epoch: {}\n\n'.format(i+1), generate_text(model, start_string=u"Hi "))
        print(' -----------------------------------------\n')

In [ ]:
print(print_multi_epoch_output(model, EPOCHS))

In [ ]:
model.load_weights('./training_checkpoints_final/ckpt_10')

model.build(tf.TensorShape([1, None]))

print('Model output from epoch: 10\n', generate_text(model, start_string=u"Hi "))
print(' -----------------------------------------\n')

In [ ]:
## Word based model

In [ ]:
print ('Length of text: {} words'.format(len(set(text.split()))))

In [ ]:
words = sorted(set(text.split()))

In [ ]:
words